<a href="https://colab.research.google.com/github/aetev/Learning-stuff-/blob/main/catagotical%20prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.lib.stride_tricks import sliding_window_view
import tracemalloc

import tensorflow_probability as tfp

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Concatenate, GaussianNoise
from tensorflow.keras.regularizers import l2


### <font color='purple'><center>**-------------------Create Data-------------------**</font>

In [ ]:
end_date = datetime.now()
start_date = end_date - timedelta(days=729)

tickers = [
    'EURUSD=X', 'USDEUR=X',  # Euro/US Dollar
    'USDJPY=X', 'JPYUSD=X',  # US Dollar/Japanese Yen
    'GBPUSD=X', 'USDGBP=X',  # British Pound/US Dollar
    'USDCHF=X', 'CHFUSD=X',  # US Dollar/Swiss Franc
    'AUDUSD=X', 'USDAUD=X',  # Australian Dollar/US Dollar
    'USDCAD=X', 'CADUSD=X',  # US Dollar/Canadian Dollar
    'NZDUSD=X', 'USDNZD=X',  # New Zealand Dollar/US Dollar
]

data_list = []
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date, interval='1h')
    data_list.append(data)

[*********************100%%**********************]  1 of 1 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['EURUSD=X']: AmbiguousTimeError("Cannot infer dst time from 2023-10-29 01:50:10, try using the 'ambiguous' argument")



[*********************100%%**********************]  1 of 1 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['USDEUR=X']: AmbiguousTimeError("Cannot infer dst time from 2023-10-29 01:50:10, try using the 'ambiguous' argument")



[*********************100%%**********************]  1 of 1 completed

...

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['USDGBP=X']: AmbiguousTimeError("Cannot infer dst time from 2023-10-29 01:50:10, try using the 'ambiguous' argument")



[*********************100%%**********************]  1 of 1 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['USDCHF=X']: AmbiguousTimeError("Cannot infer dst time from 2023-10-29 01:50:10, try using the 'ambiguous' argument")



[*********************100%%**********************]  1 of 1 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['CHFUSD=X']: AmbiguousTimeError("Cannot infer dst time from 2023-10-29 01:50:10, try using the 'ambiguous' argument")


In [ ]:
def create_data(raw_data, window_size):
    High = raw_data['High'].to_numpy()
    Close = raw_data['Close'].to_numpy()
    Low = raw_data['Low'].to_numpy()
    Std = np.std(np.diff(Close))
    Close_Diff = np.diff(Close)/Std
    High_Adj = (High[1:]-Close[1:])/Std
    Low_Adj = (Close[1:]-Low[1:])/Std

    stack = np.stack((Close_Diff, High_Adj, Low_Adj), axis=1)

    window_view = np.lib.stride_tricks.sliding_window_view(stack, (window_size,),axis=0)

    Forex_Data = window_view[:-1]
    Forex_Data = Forex_Data.astype('float32')
    target = Close_Diff[window_size:]

    return Forex_Data, target


In [ ]:
window_list = []
target_list = []
for data in data_list:
    window, target =  create_data(data, 50)
    window_list.append(window)
    target_list.append(target)


In [ ]:
# Assume data is your numpy array and bins is a numpy array defining your bins
data = target_list[0]
data = np.append(data,target_list[1])
data = np.append(data,target_list[2])

input_data = window_list[0]
input_data = np.append(input_data,+window_list[1],axis=0)
input_data = np.append(input_data,+window_list[2],axis=0)
bin_num = 51
space = .25
bins = np.linspace(-space, space, bin_num-1)
digitized = np.digitize(data, bins, right=False)
digitized = tf.keras.utils.to_categorical(digitized)
shape = digitized.shape[1]

In [ ]:
print(data)
print(input_data.shape)
print(shape)

In [ ]:
from keras.optimizers import Adam

Drop_val=.5
# Initialize the model
model = Sequential()

# Add the LSTM layer
model.add(LSTM(128,dropout=Drop_val,recurrent_dropout=Drop_val))

model.add(BatchNormalization())

model.add(Dense(52,activation='gelu'))

model.add(BatchNormalization())

model.add(Dropout(Drop_val))

# Add the Dense layer
model.add(Dense(shape, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Train the model
model.fit(input_data, digitized, epochs=200, batch_size=32,validation_split=.2)

In [ ]:
pred = model.predict(input_data)
num = 0

In [ ]:
num += 1
dist = pred[num]
print()
val = target_list[0][num]

values = np.linspace(-space, space, shape)

#round the numbers to one decimal place and only include every other value
values2 = [round(value, 2) for value in values[::5]]




bin_centers = (values[:-1] + values[1:]) / 2
dist2 = dist[:len(bin_centers)]


# Calculate the mean of the distribution
mean = np.sum(dist2 * bin_centers)


print(mean)
print(shape)
print(val)
plt.bar(range(shape), dist)
# Create a line plot on the bar chart

plt.xticks(range(len(dist))[::5], values2)

plt.plot